In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import backend as K
from random import shuffle
from statistics import mean

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/isarcasm/train.En.csv")[["tweet", "sarcastic"]]

dataset = dataset.dropna(axis = 0)
dataset.reset_index(drop=True, inplace=True)

dataset.info()
print(dataset.iloc[1062])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3467 entries, 0 to 3466
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      3467 non-null   object
 1   sarcastic  3467 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 54.3+ KB
tweet        Vaccine dose 1. Thank you, science.
sarcastic                                      0
Name: 1062, dtype: object


In [ ]:
dataset.sarcastic.value_counts()

0    2600
1     867
Name: sarcastic, dtype: int64

In [ ]:
dataset.head()

,tweet,sarcastic
0,The only thing I got from college is a caffein...,1
1,I love it when professors draw a big question ...,1
2,Remember the hundred emails from companies whe...,1
3,Today my pop-pop told me I was not “forced” to...,1
4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1


In [ ]:
X_data = dataset.tweet
Y_data = dataset.sarcastic

In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 150
trunc_type = 'post'


tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X_data)
sequences = tokenizer.texts_to_sequences(X_data)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

X = padded
Y = Y_data

# LSTM

In [ ]:
model_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_lstm.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 16)           160000    
_________________________________________________________________
bidirectional (Bidirectional (None, 32)                4224      
_________________________________________________________________
dense (Dense)                (None, 6)                 198       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 164,429
Trainable params: 164,429
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True)

fold_no = 1
for train, test in kfold.split(X):
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  class_weights = {1:2, 0:0.67}
  train = train.tolist()
  test = test.tolist()
  shuffle(test)
  shuffle(train)

  model_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model_lstm.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', f1_m])
  
  history = model_lstm.fit(X[train], Y[train], batch_size=32, epochs=5, validation_data=(X[test], Y[test]), class_weight=class_weights,shuffle=True)
  
  fold_no = fold_no + 1
  
  

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/5
87/87 [==============================] - 14s 115ms/step - loss: 0.6939 - accuracy: 0.5568 - f1_m: 0.2271 - val_loss: 0.6973 - val_accuracy: 0.3689 - val_f1_m: 0.4131
Epoch 2/5
87/87 [==============================] - 9s 103ms/step - loss: 0.6485 - accuracy: 0.6221 - f1_m: 0.5038 - val_loss: 0.6691 - val_accuracy: 0.6095 - val_f1_m: 0.3867
Epoch 3/5
87/87 [==============================] - 9s 104ms/step - loss: 0.3682 - accuracy: 0.8583 - f1_m: 0.7393 - val_loss: 0.6658 - val_accuracy: 0.6830 - val_f1_m: 0.3875
Epoch 4/5
87/87 [==============================] - 9s 103ms/step - loss: 0.1427 - accuracy: 0.9513 - f1_m: 0.9018 - val_loss: 0.8866 - val_accuracy: 0.6225 - val_f1_m: 0.3765
Epoch 5/5
87/87 [==============================] - 9s 104ms/step - loss: 0.0609 - accuracy: 0.9816 - f1_m: 0.9616 - val_loss: 0.9574 - val_accuracy: 0.6931 - val_f1_m: 0.3395
---------------------------

In [ ]:
fold_no = 1
model_with_weights_f1s = list()

for train, test in kfold.split(X):
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  train = train.tolist()
  test = test.tolist()
  shuffle(test)
  shuffle(train)

  model_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])


  model_lstm.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', f1_m])

  class_weights = {1:2, 0:0.67}
  history = model_lstm.fit(X[train], Y[train], batch_size=32, epochs=10, class_weight=class_weights,shuffle=True)
  
  scores = model_lstm.evaluate(X[test], Y[test], verbose=0)
  print(f'Score for fold {fold_no}: \n{model_lstm.metrics_names[0]} of {scores[0]};\n {model_lstm.metrics_names[1]} of {scores[1]*100}%;\n{model_lstm.metrics_names[2]} of {scores[2]};')
  model_with_weights_f1s.append(scores[2])
  fold_no = fold_no + 1

print(f'average of folds f1s is : {mean(model_with_weights_f1s)}')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
87/87 [==============================] - 12s 91ms/step - loss: 0.6959 - accuracy: 0.7432 - f1_m: 0.0292
Epoch 2/10
87/87 [==============================] - 8s 92ms/step - loss: 0.6790 - accuracy: 0.7674 - f1_m: 0.2010
Epoch 3/10
87/87 [==============================] - 8s 92ms/step - loss: 0.5654 - accuracy: 0.8536 - f1_m: 0.6294
Epoch 4/10
87/87 [==============================] - 8s 92ms/step - loss: 0.4147 - accuracy: 0.9275 - f1_m: 0.8416
Epoch 5/10
87/87 [==============================] - 8s 92ms/step - loss: 0.3308 - accuracy: 0.9607 - f1_m: 0.9186
Epoch 6/10
87/87 [==============================] - 8s 92ms/step - loss: 0.2937 - accuracy: 0.9744 - f1_m: 0.9467
Epoch 7/10
87/87 [==============================] - 8s 91ms/step - loss: 0.2702 - accuracy: 0.9784 - f1_m: 0.9587
Epoch 8/10
87/87 [==============================] - 8s 92ms/step - loss: 0.2473 - accuracy: 0.9805 - f1_

In [ ]:
fold_no = 1
model_without_weights_f1s = list()

for train, test in kfold.split(X):
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  train = train.tolist()
  test = test.tolist()
  shuffle(test)
  shuffle(train)

  model_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])


  model_lstm.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', f1_m])

  history = model_lstm.fit(X[train], Y[train], batch_size=32, epochs=10,shuffle=True)
  
  scores = model_lstm.evaluate(X[test], Y[test], verbose=0)
  print(f'Score for fold {fold_no}: \n{model_lstm.metrics_names[0]} of {scores[0]};\n {model_lstm.metrics_names[1]} of {scores[1]*100}%;\n{model_lstm.metrics_names[2]} of {scores[2]};')
  model_without_weights_f1s.append(scores[2])
  fold_no = fold_no + 1

print(f'average of folds f1s is : {mean(model_without_weights_f1s)}')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
87/87 [==============================] - 12s 90ms/step - loss: 0.5953 - accuracy: 0.7479 - f1_m: 0.0000e+00
Epoch 2/10
87/87 [==============================] - 8s 91ms/step - loss: 0.5564 - accuracy: 0.7479 - f1_m: 0.0000e+00
Epoch 3/10
87/87 [==============================] - 8s 90ms/step - loss: 0.4749 - accuracy: 0.7660 - f1_m: 0.1260
Epoch 4/10
87/87 [==============================] - 8s 91ms/step - loss: 0.2432 - accuracy: 0.9228 - f1_m: 0.8272
Epoch 5/10
87/87 [==============================] - 8s 90ms/step - loss: 0.1105 - accuracy: 0.9686 - f1_m: 0.9338
Epoch 6/10
87/87 [==============================] - 8s 90ms/step - loss: 0.0578 - accuracy: 0.9852 - f1_m: 0.9666
Epoch 7/10
87/87 [==============================] - 8s 90ms/step - loss: 0.0296 - accuracy: 0.9931 - f1_m: 0.9858
Epoch 8/10
87/87 [==============================] - 8s 91ms/step - loss: 0.0171 - accuracy: 0.99